# High Dimensional Weighted Gene Correlation Network Analysis (hdWGCNA)

**Authorship:**
Adam Klie, *08/24/2022*
***
**Description:**
Notebook to run a WGCNA on stimulated pancreatic islet multiome data (well just the RNA)
***

In [12]:
# Conversion libraries and Seurat
library(SeuratDisk)
library(SeuratData)
library(Seurat)
library(Signac)

# plotting and data science packages
library(tidyverse)
library(cowplot)
library(patchwork)

# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

In [2]:
# File names for saving
DIR = "/cellar/users/aklie/projects/igvf/beta_cell_networks/bin/infer_grns/hdwgcna/combined"
RDS_FILE = file.path(DIR, "25_neighbors_hdWGCNA_RNA_features.rds")
OUT= file.path(DIR, "analysis", "25_neighbors_hdWGCNA_RNA_features")
OUT

[1] "/cellar/users/aklie/projects/igvf/beta_cell_networks/bin/infer_grns/hdwgcna/combined/analysis/25_neighbors_hdWGCNA_RNA_features"

In [3]:
# load the preprocessed RDS from 1_network_construction.ipynb
seurat_obj <- readRDS(RDS_FILE)

In [4]:
table(seurat_obj$Sample)


primary_ND    sc_ctrl 
     14379       3473 

# Basic Visualization

In [5]:
# make a featureplot of hMEs for each module
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleFeaturePlot.png", OUT), widt=1200, height=1200)
plot_list <- ModuleFeaturePlot(
  seurat_obj,
  reduction="umap",
  features='MEs', # plot the hMEs
  order=TRUE # order so the points with highest hMEs are on top
)

# stitch together with patchwork
wrap_plots(plot_list, ncol=3)
dev.off()

[1] "brown"
[1] "yellow"
[1] "blue"
[1] "green"
[1] "turquoise"
[1] "red"


png 
  2

In [6]:
# plot module correlagram
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleCorrelation.png", OUT), widt=1200, height=1200)
ModuleCorrelogram(seurat_obj)
dev.off()

Warning message in pNew > sig.level:
“longer object length is not a multiple of shorter object length”


png 
  2

# Trait correlations

## Set-up columns in meta data for plotting

In [7]:
# Add in binary comparison columns for treatments
seurat_obj@meta.data <- cbind(
    seurat_obj@meta.data, 
    binarizeCategoricalVariable(
        seurat_obj$Sample, 
        levelOrder = c("primary_ND", "sc_ctrl"),
    )
)
colnames(seurat_obj@meta.data)

[1] "orig.ident"                  "nCount_RNA"                 
 [3] "nFeature_RNA"                "percent.mt"                 
 [5] "nCount_RNA_raw"              "nFeature_RNA_raw"           
 [7] "nCount_ATAC"                 "nFeature_ATAC"              
 [9] "nCount_SCT"                  "nFeature_SCT"               
[11] "SCT.weight"                  "ATAC.weight"                
[13] "wsnn_res.0.2"                "seurat_clusters"            
[15] "log_nCount_ATAC"             "log_nCount_SCT"             
[17] "log_nFeature_ATAC"           "log_nFeature_SCT"           
[19] "nCount_mpeak"                "nFeature_mpeak"             
[21] "batch"                       "condition"                  
[23] "time_point"                  "sample"                     
[25] "atac_raw_reads"              "atac_mitochondrial_reads"   
[27] "atac_fragments"              "atac_pct.mt"                
[29] "TSS.enrichment"              "TSS.percentile"             
[31] "mpeak.weight"                "wsnn_res.0.3"               
[33] "nCount_peaks"                "nFeature_peaks"             
[35] "peaks.weight"                "wsnn_res.0.5"               
[37] "nCount_peak"                 "nFeature_peak"              
[39] "cell.type.1"                 "cell.type.2"                
[41] "Sample"                      "log_count_RNA"              
[43] "count_rank_RNA"              "log_count_ATAC"             
[45] "count_rank_ATAC"             "nucleosome_signal"          
[47] "nucleosome_percentile"       "donor"                      
[49] "scrub_score"                 "scrub_pred"                 
[51] "BGMM_prob"                   "BGMM_pred"                  
[53] "SCT_snn_res.0.1"             "nCount_cpeak"               
[55] "nFeature_cpeak"              "cpeak.weight"               
[57] "wsnn_res.0.1"                "disease"                    
[59] "percent.rps"                 "percent.rpl"                
[61] "integrated.rpca_snn_res.0.8" "experiment"                 
[63] "poly.hormone"                "integrated.rpca_snn_res.0.3"
[65] "activity.prediction"         "activity.prediction.score"  
[67] "rna.anno"                    "prediction.qaulity"         
[69] "nCount_window"               "nFeature_window"            
[71] "integrated.rpca.weight"      "window.weight"              
[73] "wsnn_res.0.03"               "nCount_unionpeaks"          
[75] "nFeature_unionpeaks"         "unionpeaks.weight"          
[77] "wsnn_res.0.01"               "integrated.rpca_snn_res.0.1"
[79] "samp"                        "integrated_snn_res.0.1"     
[81] "integrated_snn_res.0.2"      "integrated_snn_res.0.01"    
[83] "integrated_snn_res.0.02"     "integrated_snn_res.0.005"   
[85] "integrated_snn_res.0.4"      "integrated_snn_res.0.3"     
[87] "manual_clusters"             "hdwgcna_groups"             
[89] "metacell_grouping"           "sc_ctrl.vs.primary_ND"

In [8]:
# Double check these, they should match up. Note that untreated is equivalent to 1 here
print(table(seurat_obj$Sample))
print(table(seurat_obj$sc_ctrl.vs.primary_ND))


primary_ND    sc_ctrl 
     14379       3473 

    0     1 
14379  3473 


## Plot correlation heatmaps

### Cell type correlations

In [9]:
# list of traits to correlate for cell type
cur_traits <- c('sc_ctrl.vs.primary_ND')
seurat_obj <- ModuleTraitCorrelation(
  seurat_obj,
  features="MEs",
  traits = cur_traits,
)

In [10]:
# Grabe the correlation table and eventually save it
mt_cor <- GetModuleTraitCorrelation(seurat_obj)
print(names(mt_cor))
mt_cor$cor$all_cells

[1] "cor"  "pval" "fdr" 


yellow        blue   turquoise       green       brown         red 
 0.81554656  0.59969849  0.54169629  0.66310254 -0.01373954  0.49380592

In [11]:
print(mt_cor$cor$all_cells)

     yellow        blue   turquoise       green       brown         red 
 0.81554656  0.59969849  0.54169629  0.66310254 -0.01373954  0.49380592 


In [77]:
# Grab and order the correlation dataframe to match the built-in function ordering
corr_df <- mt_cor$cor$all_cells
corr_df <- corr_df[, 1:3]

ERROR: Error in corr_df[, 1:3]: incorrect number of dimensions


In [79]:
# Display the correlation values within a heatmap plot
options(repr.plot.width=16, repr.plot.height=8)
png(sprintf("%s_ModuleCelltypeCorrelationsAllCells.png", OUT), widt=1200, height=800)
par(mar=c(10,6,4,1))
textMatrix = paste(
    signif(corr_df, 2), sep="")
dim(textMatrix) = dim(corr_df)
labeledHeatmap(
    Matrix=corr_df,
    xLabels=colnames(corr_df),
    yLabels=rownames(corr_df),
    ySymbols=rownames(corr_df),
    textMatrix=textMatrix,
    colorLabels=FALSE,
    colors=blueWhiteRed(50),
    cex.text=1.2,
    cex.lab.x=1,
    cex.lab.y=1,
    zlim=c(-1, 1),
    main=paste("Module-Cell type relationships"),
)
dev.off()

ERROR: Error in if (is.null(yLabels) & (!is.null(xLabels)) & (dim(Matrix)[1] == : argument is of length zero


In [37]:
# Plot a heatmap of p-values for correlations across cell-types
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleCelltypeCorrelationFDRValues.png", OUT), widt=1000, height=1200)
PlotModuleTraitCorrelation(
  seurat_obj,
  label = 'fdr',
  label_symbol = 'numeric',
  text_size = 5,
  text_digits = 3,
  combine=TRUE,
  plot_max = 0.4
)
dev.off()

ERROR: Error in PlotModuleTraitCorrelation(seurat_obj, label = "fdr", label_symbol = "numeric", : ModuleTraitCorrelation was run only for one trait. Heatmaps are not suggested for visualizing only one variable!


In [22]:
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleCelltypeCorrelationFDRStars.png", OUT), widt=1000, height=1200)
PlotModuleTraitCorrelation(
  seurat_obj,
  label = 'fdr',
  label_symbol = 'stars',
  text_size = 5,
  text_digits = 3,
  combine=TRUE,
  plot_max = 0.4
)
dev.off()

[1] "all_cells"
[1] "data.SC.beta.1" "data.SC.beta.2" "data.SC.beta.3"
[1] "SC.beta"
[1] "data.SC.beta.1" "data.SC.beta.2" "data.SC.beta.3"


png 
  2

# Module analysis

In [80]:
modules <- GetModules(seurat_obj)

In [81]:
write.table(modules, sprintf("%s_modules.tsv", OUT), sep="\t")

# Motif analysis

In [13]:
# packages for TF motif analysis
library(JASPAR2020)
library(motifmatchr)
library(TFBSTools)
library(GenomicRanges)
library(EnsDb.Hsapiens.v86)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘package:tidyr’:

    expand


The following objects are masked from ‘pac

In [14]:
MotifScan <- function(
  seurat_obj,
  species_genome, # hg38, mm10, etc...
  pfm, # matrix set from JASPAR2020 for example
  EnsDb, # Ensembl database such as EnsDb.Mmusculus.v79
  wgcna_name=NULL
){

  if(is.null(wgcna_name)){wgcna_name <- seurat_obj@misc$active_wgcna}

  # get a dataframe of just the motif name and the motif ID:
  motif_df <- data.frame(
    motif_name = purrr::map(1:length(pfm), function(i){pfm[[i]]@name}) %>% unlist,
    motif_ID = purrr::map(1:length(pfm), function(i){pfm[[i]]@ID}) %>% unlist
  )

  # get promoters of protein coding genes from the given Ensdb
  # note: everything breaks if I try to use X & Y chromosomes.
  gene.promoters <- ensembldb::promoters(EnsDb, filter = ~ gene_biotype == "protein_coding") %>%
    subset(seqnames %in% c(1:100))
  gene.coords <- ensembldb::genes(EnsDb, filter = ~ gene_biotype == "protein_coding") %>%
    subset(seqnames %in% c(1:100))

  # add the gene name to the promoter object
  gene.promoters$symbol <- gene.coords$symbol[match(gene.promoters$gene_id, names(gene.coords))]

  # drop unnecessary chromosomes
  gene.promoters <- keepSeqlevels(gene.promoters, value= levels(droplevels(seqnames(gene.promoters))))

  # rename seqlevels to add 'chr', remove X&Y chromosomes because they break everything
  old_levels <- levels(seqnames(gene.promoters))
  new_levels <- ifelse(old_levels %in% c('X', 'Y'), old_levels, paste0('chr', old_levels))
  gene.promoters <- renameSeqlevels(gene.promoters, new_levels)

  # set the genome (not sure if we NEED to do this...)
  genome(seqinfo(gene.promoters)) <- species_genome

  # set up promoters object that only has the necessary info for motifmatchr
  my_promoters <- GRanges(
    seqnames =  droplevels(seqnames(gene.promoters)),
    IRanges(
      start = start(gene.promoters),
      end = end(gene.promoters)
    ),
    symbol = gene.promoters$symbol,
    genome=species_genome
  )

  # scan these promoters for motifs:
  print('Matching motifs...')
  motif_ix <- motifmatchr::matchMotifs(pfm, my_promoters, genome=species_genome)

  # get the matches
  tf_match <- motifmatchr::motifMatches(motif_ix)
  rownames(tf_match) <- my_promoters$symbol

  # use motif names as the column names:
  colnames(tf_match) <- motif_df$motif_name

  # only keep genes that are in the Seurat object and in the given EnsDb:
  gene_list <- rownames(seurat_obj)
  gene_list <- gene_list[gene_list %in% rownames(tf_match)]
  tf_match <- tf_match[gene_list,]

  # get list of target genes for each TF:
  print('Getting TF target genes...')
  tfs <- motif_df$motif_name
  tf_targets <- list()
  n_targets <- list()
  for(cur_tf in tfs){
    tf_targets[[cur_tf]] <- names(tf_match[,cur_tf][tf_match[,cur_tf]])
    n_targets[[cur_tf]] <- length(tf_targets[[cur_tf]] )
  }
  n_targets <- unlist(n_targets)

  # add number of target genes to motif df
  motif_df$n_targets <- n_targets

  # add info to seurat object
  seurat_obj <- SetMotifMatrix(seurat_obj, tf_match)
  seurat_obj <- SetMotifs(seurat_obj, motif_df)
  seurat_obj <- SetMotifTargets(seurat_obj, tf_targets)
  seurat_obj <- SetPFMList(seurat_obj, pfm)

  seurat_obj
}

In [15]:
#' Displays the top n TFs in a set of modules as a bar plot
#'
#' @param seurat_obj A Seurat object
#' @param wgcna_name The name of the hdWGCNA experiment in the seurat_obj@misc slot
#' @keywords scRNA-seq
#' @export
#' @examples
#' MotifOverlapBarPlot
MotifOverlapBarPlot <- function(
  seurat_obj,
  n_tfs = 10,
  plot_size = c(5,6),
  outdir = "MotifOverlaps/",
  motif_font = 'helvetica_regular',
  module_names = NULL,
  wgcna_name=NULL
){

  if(is.null(wgcna_name)){wgcna_name <- seurat_obj@misc$active_wgcna}

  # make output dir if it doesn't exist:
  if(!dir.exists(outdir)){dir.create(outdir)}

  # get Modules
  modules <- GetModules(seurat_obj)
  mods <- levels(modules$module)
  mods <- mods[mods != 'grey']

  if(is.null(module_names)){module_names <- mods}

  # get overlap info from Seurat obj
  overlap_df <- GetMotifOverlap(seurat_obj, wgcna_name)
  motif_df <- GetMotifs(seurat_obj)

  # get pfm list from Seurat obj
  pfm <- GetPFMList(seurat_obj)

  # add motif ID to the overlap_df
  overlap_df$motif_ID <- motif_df$motif_ID[match(overlap_df$tf, motif_df$motif_name)]

  # subset by the modules that we are using:
  overlap_df <- overlap_df %>% subset(module %in% module_names)

  for(cur_mod in module_names){
    print(cur_mod)
    # get df for cur mod
    plot_df <- overlap_df %>% subset(module == cur_mod) %>% top_n(n_tfs, wt=odds_ratio) %>%
      arrange(desc(odds_ratio))

    # make the barplot
    p1 <- plot_df %>% ggplot(aes(y=reorder(tf, odds_ratio), fill=odds_ratio, x=odds_ratio)) +
      geom_bar(stat='identity', width=0.7) + NoLegend() +
      scale_fill_gradient(high=unique(plot_df$color), low='grey90') +
      ylab('') + theme(
        axis.line.y = element_blank(),
        axis.text.y = element_blank(),
        plot.margin = margin(
          t = 0, r = 0, b = 0, l = 0
        )
      )

    # make the motif logo plots:
    plot_list <- list()
    for(i in 1:nrow(plot_df)){
      cur_id <- plot_df[i,'motif_ID']
      cur_name <- plot_df[i,'tf']
      plot_list[[cur_id]] <- ggplot() +
        ggseqlogo::geom_logo( as.matrix(pfm[[cur_id]]), font=motif_font) +
        ggseqlogo::theme_logo() +
        xlab('') + ylab(cur_name) + theme(
          axis.text.x=element_blank(),
          axis.text.y=element_blank(),
          axis.title.y = element_text(angle=0),
          plot.margin = margin(t = 0,  # Top margin
                                 r = 0,  # Right margin
                                 b = 0,  # Bottom margin
                                 l = 0) # Left margin
        )
    }

    # wrap the motif logos
    patch1 <- wrap_plots(plot_list, ncol=1)

    # assemble the plot with patchwork
    outplot <- (patch1 | p1) +
          plot_layout(ncol=2, widths=c(1,2)) +
          plot_annotation(title=paste0('Motif overlaps with ', cur_mod),
          theme = theme(plot.title=element_text(hjust=0.5))
        )

    pdf(paste0(outdir, '/', cur_mod, '_motif_overlaps.pdf'), width=plot_size[1], height=plot_size[2], useDingbats=FALSE)
    print(outplot)
    dev.off()

  }

}

In [16]:
# get the pfm from JASPAR2020 using TFBSTools
pfm_core <- TFBSTools::getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)

In [17]:
# run the motif scan with these settings for the mouse dataset
seurat_obj <- MotifScan(
  seurat_obj,
  EnsDb = EnsDb.Hsapiens.v86,
  species_genome = 'hg38',
  pfm = pfm_core
)
dim(GetMotifMatrix(seurat_obj))

[1] "Matching motifs..."



Attaching package: 'XVector'


The following object is masked from 'package:purrr':

    compact



Attaching package: 'Biostrings'


The following object is masked from 'package:base':

    strsplit




[1] "Getting TF target genes..."


[1] 17816   746

In [18]:
GetMotifMatrix(seurat_obj)[1:6, 1:6]

6 x 6 sparse Matrix of class "lgCMatrix"
       Arnt Ahr::Arnt Ddit3::Cebpa Mecom FOXF2 FOXD1
OR4F5     .         .            .     |     |     .
OR4F29    .         .            |     |     .     .
OR4F16    .         .            |     |     .     .
SAMD11    .         |            |     .     .     .
NOC2L     |         .            .     .     .     .
KLHL17    .         |            .     .     .     |

In [19]:
# TF target genes
target_genes <- GetMotifTargets(seurat_obj)

# check target genes for one TF:
head(target_genes$SOX9)

[1] "OR4F5"  "TTLL10" "SDF4"   "ATAD3C" "MIB2"   "CDK11B"

In [20]:
# TestGeneOverlap
library(GeneOverlap)


In [21]:
# overlap between modules & TF target genes:
seurat_obj<- OverlapModulesMotifs(seurat_obj)

# look at the overlap data
head(GetMotifOverlap(seurat_obj))

,module,tf,color,odds_ratio,pval,fdr,Significance,Jaccard,size_intersection
,<fct>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Arnt,yellow,Arnt,yellow,2.193799,5.778874e-06,1.452344e-05,***,0.009727626,50
Ahr::Arnt,yellow,Ahr::Arnt,yellow,2.573329,1.627436e-09,6.268848e-09,***,0.010409031,71
Ddit3::Cebpa,yellow,Ddit3::Cebpa,yellow,1.368993,7.753771e-02,9.817788e-02,,0.006720742,29
Mecom,yellow,Mecom,yellow,1.677371,3.047744e-03,5.153647e-03,**,0.007902164,42
FOXF2,yellow,FOXF2,yellow,1.442171,4.128123e-02,5.497614e-02,,0.007022164,32
FOXD1,yellow,FOXD1,yellow,1.480708,4.858949e-02,6.383521e-02,,0.007175660,25


In [22]:
motif_overlap <- GetMotifOverlap(seurat_obj)

In [23]:
write.table(motif_overlap, sprintf("%s_motif_overlap.tsv", OUT), sep="\t")

In [32]:
# plot the top TFs overlapping with
MotifOverlapBarPlot(
  seurat_obj,
  #motif_font = 'xkcd_regular',
  outdir = 'analysis/MotifOverlaps/',
  plot_size=c(5,6)
)

[1] "yellow"


Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = F

[1] "turquoise"


Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = F

[1] "black"


...

Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = F

[1] "blue"


Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = F

[1] "red"


Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead."
Warning message:
"`guides(<scale> = F

---

# Scratch